In [1]:
from siftdetector import detect_keypoints
%matplotlib inline
from numpy_sift import SIFTDescriptor 
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '/home/ubuntu/dev/opencv-3.1/build/lib')
import cv2
import time
import numpy as np
import cv2
from sklearn.cluster import KMeans
import os
from math import log
from tqdm import tqdm

In [18]:
def return_labels(directoryList, range_size, added_limit, count_limit, patch_size=32):
    SD = SIFTDescriptor(patchSize = patch_size)

    #creates initial vector of size 3968
    sift_pictures = np.asarray([[0 for _ in range(range_size)]])
    for directory in directoryList:
        #go through the directories with the relevant images
        for filename in os.listdir(directory):
            name = directory + '/' + filename
            print name
            if name[-3:] == 'png' or name[-3:] == 'jpg':
                image = cv2.imread(name)
                
                #transform it to black and white and get features
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                h, w = gray.shape
                corners = cv2.goodFeaturesToTrack(gray,count_limit,0.001,10)

                corners = np.int0(corners)
                image_sift_features = np.asarray([]) 
                count, actually_added = 0, 0
                
                #Here we want to get 31 images for our feature vector and then break out of the loop
                while actually_added < added_limit and count < count_limit:
                    corner = corners[count][0]
                    if patch_size/2 <= corner[1] <= h-patch_size/2 and patch_size/2 <= corner[0] <= w-patch_size/2:
                        patch = gray[corner[1]-patch_size/2:corner[1]+patch_size/2, corner[0]-patch_size/2:corner[0]+patch_size/2]
                        sift = SD.describe(patch)
                        image_sift_features = np.append(image_sift_features, sift)
                        actually_added += 1
                    count += 1

                sift_pictures = np.append(sift_pictures, [image_sift_features], axis=0)

    return sift_pictures[1:]



In [22]:
sift_pictures = return_labels(['Auditorium', 'Bowling'], 2304, 18, 100)
kmeans = KMeans(n_clusters=2, random_state=1).fit(sift_pictures)
test_pictures = return_labels(['test'], 3968, 31, 100)
print kmeans.predict(sift_pictures)
print kmeans.predict(test_pictures)

Auditorium/.DS_Store
Auditorium/00000001.jpg
Auditorium/00000003.jpg
Auditorium/00000004.jpg
Auditorium/00000008.jpg
Auditorium/00000009.jpg
Auditorium/00000011.jpg
Auditorium/00000016.jpg
Auditorium/00000022.jpg
Auditorium/00000024.jpg
Auditorium/00000025.jpg
Auditorium/00000026.jpg
Auditorium/00000030.jpg
Auditorium/00000031.jpg
Auditorium/00000032.jpg
Auditorium/00000049.jpg
Auditorium/00000062.jpg
Bowling/.DS_Store
Bowling/00000006.jpg
Bowling/00000010(1).jpg
Bowling/00000011.jpg
Bowling/00000012(1).jpg
Bowling/00000026(1).jpg
Bowling/00000029(1).jpg
Bowling/00000031.jpg
Bowling/00000044.jpg
Bowling/00000055(1).jpg
Bowling/00000058.jpg
Bowling/00000059.jpg
Bowling/00000072.jpg
Bowling/00000095.jpg
Bowling/00000098.jpg
Bowling/00000104.jpg
Bowling/00000126(1).jpg
Bowling/00000129(1).jpg
Bowling/00000134(1).jpg
test/.DS_Store
test/00000002.jpg
test/00000017.jpg
test/00000162.jpg
test/00000190.jpg
test/00000510.jpg
test/00000591.jpg
[1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1

ValueError: Incorrect number of features. Got 3968 features, expected 2304